# Permissions

> Flexible, transparent role-based access control (RBAC) system for FastHTML applications

In [ ]:
#| default_exp permissions

## Imports and utils

In [ ]:
#| export
from typing import Optional, Dict, Any, List, Union, Callable, Set
from functools import wraps
from fasthtml.common import *
from launch_kit.auth import get_user_from_session
import inspect

In [ ]:
#| export
__all__ = ['require_auth', 'require_role', 'require_permission', 'auth_required', 'role_required', 'permission_required',
           'get_user_permissions', 'has_permission', 'check_role_hierarchy', 'register_permission', 'get_permissions_for_role',
           'set_role_permissions', 'add_role_permission', 'remove_role_permission', 'clear_permission_cache',
           'ROLE_HIERARCHY', 'ROLE_PERMISSIONS', 'PERMISSION_DESCRIPTIONS']

In [ ]:
#| hide
from nbdev.showdoc import *

## Quick Start

Launch Kit's permissions module provides simple, transparent access control for your FastHTML applications:

```python
from launch_kit.permissions import *
from fasthtml.common import *

# 1. Protect routes with decorators
@rt("/admin")
@auth_required
def get(req, sess):
    return "Welcome to admin area!"

# 2. Check specific roles
@rt("/moderator/dashboard")
@role_required("moderator")
def get(req, sess):
    return "Moderator Dashboard"

# 3. Check granular permissions
@rt("/users/delete")
@permission_required("delete_users")
def post(req, sess):
    # Delete user logic
    pass

# 4. Manual permission checks
@rt("/api/sensitive")
def get(req, sess):
    if not require_permission("view_sensitive_data", req, sess):
        return JSONResponse({"error": "Forbidden"}, status_code=403)
    return {"data": "sensitive information"}
```

That's it! Your routes are now protected with role-based access control.

## Overview

This module provides a complete RBAC (Role-Based Access Control) system:

### Core Functions

| Function | Purpose | When to Use |
|----------|---------|-------------|
| `require_auth` | Check if user is authenticated | Manual auth checks |
| `require_role` | Check if user has specific role | Manual role checks |
| `require_permission` | Check if user has permission | Manual permission checks |

### Decorators

| Decorator | Purpose | When to Use |
|-----------|---------|-------------|
| `@auth_required` | Require authentication | Protect any authenticated route |
| `@role_required` | Require specific role | Admin/moderator areas |
| `@permission_required` | Require specific permission | Granular access control |

### Permission Management

| Function | Purpose | When to Use |
|----------|---------|-------------|
| `get_user_permissions` | Get all user permissions | Display user capabilities |
| `register_permission` | Register new permission | Add custom permissions |
| `set_role_permissions` | Set permissions for role | Configure roles |
| `clear_permission_cache` | Clear cached permissions | After role changes |

## Default Configuration

Launch Kit provides sensible defaults for role hierarchy and permissions:

In [ ]:
#| export
# Role hierarchy - higher number = more privileges
ROLE_HIERARCHY = {
    'user': 1,
    'moderator': 2,
    'admin': 3
}

# Default role permissions
ROLE_PERMISSIONS = {
    'user': {
        'read_own_data',
        'update_own_data',
        'delete_own_data'
    },
    'moderator': {
        'read_own_data',
        'update_own_data',
        'delete_own_data',
        'read_all_data',
        'moderate_content',
        'manage_users'
    },
    'admin': {'*'}  # All permissions
}

# Permission descriptions (optional, for documentation)
PERMISSION_DESCRIPTIONS = {
    'read_own_data': 'Read user\'s own data',
    'update_own_data': 'Update user\'s own data',
    'delete_own_data': 'Delete user\'s own data',
    'read_all_data': 'Read all users\' data',
    'moderate_content': 'Moderate user-generated content',
    'manage_users': 'Create, update, delete other users',
    'delete_users': 'Delete user accounts',
    'view_sensitive_data': 'View sensitive system data'
}

## Core Permission Functions

Simple boolean functions for checking authentication, roles, and permissions:

In [ ]:
#| export
def require_auth(req, # The FastHTML Request object
                 sess # The FastHTML Session object
                 ) -> bool: # True if user is authenticated
    """Check if user is authenticated.
    
    This is the simplest permission check - just verifies that a user is logged in.
    """
    return sess.get('auth') is not None

In [ ]:
show_doc(require_auth)

---

[source](https://github.com/LotsOfOrg/launch-kit/blob/main/launch_kit/permissions.py#L63){target="_blank" style="float:right; font-size:smaller"}

### require_auth

>      require_auth (req, sess)

*Check if user is authenticated.

This is the simplest permission check - just verifies that a user is logged in.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| req |  | The FastHTML Request object |
| sess |  | The FastHTML Session object |
| **Returns** | **bool** | **True if user is authenticated** |

In [ ]:
#| export
def check_role_hierarchy(user_role: Optional[str], # The user's current role
                        required_role: str          # The required role
                        ) -> bool:                  # True if user role >= required role
    """Check if user's role meets or exceeds the required role in hierarchy.
    
    Uses ROLE_HIERARCHY to determine if a user's role has sufficient privileges.
    For example, an 'admin' can access 'moderator' areas.
    """
    if not user_role:
        return False
    
    # Get role levels from hierarchy
    user_level = ROLE_HIERARCHY.get(user_role, 0)
    required_level = ROLE_HIERARCHY.get(required_role, float('inf'))
    
    return user_level >= required_level

In [ ]:
#| export
def require_role(role: str,  # The required role
                 req,        # The FastHTML Request object  
                 sess        # The FastHTML Session object
                 ) -> bool:  # True if user has the required role or higher
    """Check if user has the required role or higher in the hierarchy.
    
    Uses role hierarchy so admins can access moderator areas, etc.
    """
    user = get_user_from_session(sess)
    if not user:
        return False
    
    return check_role_hierarchy(user.get('role'), role)

In [ ]:
show_doc(require_role)

---

[source](https://github.com/LotsOfOrg/launch-kit/blob/main/launch_kit/permissions.py#L91){target="_blank" style="float:right; font-size:smaller"}

### require_role

>      require_role (role:str, req, sess)

*Check if user has the required role or higher in the hierarchy.

Uses role hierarchy so admins can access moderator areas, etc.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| role | str | The required role |
| req |  | The FastHTML Request object |
| sess |  | The FastHTML Session object |
| **Returns** | **bool** | **True if user has the required role or higher** |

In [ ]:
#| export
def get_user_permissions(user: Dict[str, Any] # The user dictionary from session
                        ) -> Set[str]:        # Set of permission strings
    """Get all permissions for a user based on their role.
    
    Returns a set of permission strings. Admins get '*' which means all permissions.
    """
    if not user:
        return set()
    
    role = user.get('role', 'user')
    permissions = ROLE_PERMISSIONS.get(role, set())
    
    # Return a copy to prevent modification
    return permissions.copy() if isinstance(permissions, set) else permissions

In [ ]:
#| export  
def has_permission(user: Dict[str, Any],      # The user dictionary
                   permission: str            # The permission to check
                   ) -> bool:                 # True if user has permission
    """Check if user has a specific permission.
    
    Handles the special case where admins have '*' meaning all permissions.
    """
    if not user:
        return False
    
    permissions = get_user_permissions(user)
    
    # Admin with wildcard permission
    if '*' in permissions:
        return True
    
    return permission in permissions

In [ ]:
#| export
def require_permission(permission: str,  # The required permission
                      req,              # The FastHTML Request object
                      sess              # The FastHTML Session object  
                      ) -> bool:        # True if user has permission
    """Check if user has a specific permission.
    
    This is for granular permission checking beyond roles.
    """
    user = get_user_from_session(sess)
    if not user:
        return False
    
    return has_permission(user, permission)

In [ ]:
show_doc(require_permission)

---

[source](https://github.com/LotsOfOrg/launch-kit/blob/main/launch_kit/permissions.py#L141){target="_blank" style="float:right; font-size:smaller"}

### require_permission

>      require_permission (permission:str, req, sess)

*Check if user has a specific permission.

This is for granular permission checking beyond roles.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| permission | str | The required permission |
| req |  | The FastHTML Request object |
| sess |  | The FastHTML Session object |
| **Returns** | **bool** | **True if user has permission** |

## Permission Decorators

Decorators provide a clean way to protect FastHTML routes:

In [ ]:
#| export
def auth_required(func: Callable             # The route function to protect
                 ) -> Callable:              # The wrapped function
    """Decorator that requires authentication for a route.
    
    Redirects to /login if user is not authenticated.
    Works with FastHTML route functions that accept req and sess parameters.
    
    Example:
        @rt('/dashboard')
        @auth_required
        def get(req, sess):
            return "Dashboard content"
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Find req and sess in args or kwargs
        req, sess = _extract_req_sess(func, args, kwargs)
        
        if not require_auth(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        return func(*args, **kwargs)
    
    return wrapper

In [ ]:
#| export
def role_required(role: str                   # The required role
                 ) -> Callable:               # Decorator function
    """Decorator that requires a specific role for a route.
    
    Returns 403 Forbidden if user doesn't have the required role.
    
    Example:
        @rt('/admin')
        @role_required('admin')
        def get(req, sess):
            return "Admin panel"
    """
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Find req and sess in args or kwargs
            req, sess = _extract_req_sess(func, args, kwargs)
            
            if not require_role(role, req, sess):
                # Check if user is authenticated but lacks permission
                if require_auth(req, sess):
                    return HTMLResponse(
                        "<h1>403 Forbidden</h1><p>You don't have permission to access this resource.</p>",
                        status_code=403
                    )
                else:
                    return RedirectResponse('/login', status_code=303)
            
            return func(*args, **kwargs)
        
        return wrapper
    
    return decorator

In [ ]:
#| export
def permission_required(permission: str       # The required permission
                       ) -> Callable:         # Decorator function  
    """Decorator that requires a specific permission for a route.
    
    Returns 403 Forbidden if user doesn't have the required permission.
    
    Example:
        @rt('/users/delete')
        @permission_required('delete_users')
        def post(req, sess, user_id: int):
            # Delete user logic
            pass
    """
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Find req and sess in args or kwargs
            req, sess = _extract_req_sess(func, args, kwargs)
            
            if not require_permission(permission, req, sess):
                # Check if user is authenticated but lacks permission
                if require_auth(req, sess):
                    return HTMLResponse(
                        "<h1>403 Forbidden</h1><p>You don't have permission to access this resource.</p>",
                        status_code=403
                    )
                else:
                    return RedirectResponse('/login', status_code=303)
            
            return func(*args, **kwargs)
        
        return wrapper
    
    return decorator

In [ ]:
#| export
def _extract_req_sess(func: Callable,                    # The function being decorated
                      args: tuple,                       # Positional arguments
                      kwargs: dict                       # Keyword arguments
                      ) -> tuple[Any, Any]:              # (req, sess) tuple
    """Extract req and sess from function arguments.
    
    This helper function handles the various ways FastHTML might pass req and sess.
    """
    # Get function signature
    sig = inspect.signature(func)
    params = list(sig.parameters.keys())
    
    # Try to find req and sess
    req = None
    sess = None
    
    # Check kwargs first
    if 'req' in kwargs:
        req = kwargs['req']
    if 'sess' in kwargs:
        sess = kwargs['sess']
    
    # Then check positional args
    if req is None and 'req' in params:
        req_idx = params.index('req')
        if req_idx < len(args):
            req = args[req_idx]
    
    if sess is None and 'sess' in params:
        sess_idx = params.index('sess')
        if sess_idx < len(args):
            sess = args[sess_idx]
    
    if req is None or sess is None:
        raise ValueError("Could not find req and sess parameters in function")
    
    return req, sess

## Permission Management

Functions for managing and configuring permissions:

In [ ]:
#| export
def register_permission(name: str,                        # Permission identifier
                       description: Optional[str] = None  # Human-readable description
                       ) -> None:
    """Register a new permission in the system.
    
    This is optional but helps with documentation and validation.
    """
    if description:
        PERMISSION_DESCRIPTIONS[name] = description

In [ ]:
#| export
def get_permissions_for_role(role: str        # The role name
                           ) -> Set[str]:     # Set of permissions
    """Get all permissions assigned to a role."""
    return ROLE_PERMISSIONS.get(role, set()).copy()

In [ ]:
#| export
def set_role_permissions(role: str,                              # The role name
                        permissions: Union[Set[str], List[str]]  # Permissions to assign
                        ) -> None:
    """Set permissions for a role, replacing any existing permissions."""
    ROLE_PERMISSIONS[role] = set(permissions) if isinstance(permissions, list) else permissions

In [ ]:
#| export
def add_role_permission(role: str,            # The role name
                       permission: str        # Permission to add
                       ) -> None:
    """Add a single permission to a role."""
    if role not in ROLE_PERMISSIONS:
        ROLE_PERMISSIONS[role] = set()
    ROLE_PERMISSIONS[role].add(permission)

In [ ]:
#| export  
def remove_role_permission(role: str,         # The role name
                          permission: str     # Permission to remove
                          ) -> None:
    """Remove a single permission from a role."""
    if role in ROLE_PERMISSIONS:
        ROLE_PERMISSIONS[role].discard(permission)

## Session Integration

Utilities for caching permissions in the session for performance:

In [ ]:
#| export
def clear_permission_cache(sess               # The FastHTML Session object
                         ) -> None:
    """Clear cached permissions from session.
    
    Call this after changing a user's role or permissions.
    """
    if '_permissions_cache' in sess:
        del sess['_permissions_cache']

## Examples

### Basic Authentication Protection

In [ ]:
#| eval: false
from fasthtml.common import *
from launch_kit.permissions import *

app, rt = fast_app()

# Simple authentication check
@rt('/dashboard')
@auth_required
def get(req, sess):
    user = get_user_from_session(sess)
    return Div(
        H1(f"Welcome {user['username']}!"),
        P("This is your dashboard.")
    )

### Role-Based Access Control

In [ ]:
#| eval: false
# Admin-only area
@rt('/admin')
@role_required('admin')
def get(req, sess):
    return Div(
        H1("Admin Panel"),
        P("Only administrators can see this.")
    )

# Moderator area (admins can also access)
@rt('/moderate')
@role_required('moderator')  
def get(req, sess):
    return Div(
        H1("Moderation Queue"),
        P("Moderators and admins can see this.")
    )

### Granular Permission Checks

In [ ]:
#| eval: false
# Check specific permission
@rt('/users/{user_id}/delete', methods=['POST'])
@permission_required('delete_users')
def delete_user(req, sess, user_id: int):
    # Delete user logic here
    return {"status": "deleted", "user_id": user_id}

# Manual permission check for conditional UI
@rt('/users/{user_id}')
@auth_required
def get(req, sess, user_id: int):
    user = get_user_from_session(sess)
    can_delete = has_permission(user, 'delete_users')
    
    return Div(
        H1(f"User Profile #{user_id}"),
        Button(
            "Delete User",
            hx_post=f"/users/{user_id}/delete",
            hx_confirm="Are you sure?"
        ) if can_delete else None
    )

### Custom Permission Configuration

In [ ]:
#| eval: false
# Register custom permissions
register_permission('export_data', 'Export system data to CSV')
register_permission('view_analytics', 'View analytics dashboard')

# Create a custom role
set_role_permissions('analyst', {
    'read_all_data',
    'view_analytics', 
    'export_data'
})

# Add permission to existing role
add_role_permission('moderator', 'view_analytics')

# Remove permission from role
remove_role_permission('user', 'delete_own_data')

### API Endpoints with Permissions

In [ ]:
#| eval: false
# JSON API with permission checks
@rt('/api/users')
def get(req, sess):
    if not require_permission('read_all_data', req, sess):
        return JSONResponse(
            {"error": "Forbidden", "message": "Insufficient permissions"},
            status_code=403
        )
    
    # Return user data
    return JSONResponse({"users": []})

# Using decorators with JSON responses
@rt('/api/admin/stats')
@role_required('admin')
def get(req, sess):
    # Admin-only statistics
    return JSONResponse({
        "total_users": 1000,
        "active_sessions": 42
    })

## Testing Permissions

In [ ]:
#| hide
# Test core permission functions
print("Testing core permission functions...")

# Mock session and request
mock_req = type('Request', (), {})()
mock_sess = {}

# Test require_auth
assert require_auth(mock_req, mock_sess) == False
mock_sess['auth'] = True
assert require_auth(mock_req, mock_sess) == True
print("✓ require_auth working")

# Test role hierarchy
assert check_role_hierarchy('admin', 'user') == True
assert check_role_hierarchy('admin', 'moderator') == True  
assert check_role_hierarchy('admin', 'admin') == True
assert check_role_hierarchy('moderator', 'admin') == False
assert check_role_hierarchy('user', 'moderator') == False
assert check_role_hierarchy(None, 'user') == False
print("✓ Role hierarchy working")

# Test require_role
mock_sess['user'] = {'role': 'admin'}
assert require_role('admin', mock_req, mock_sess) == True
assert require_role('moderator', mock_req, mock_sess) == True
assert require_role('user', mock_req, mock_sess) == True

mock_sess['user'] = {'role': 'moderator'}
assert require_role('admin', mock_req, mock_sess) == False
assert require_role('moderator', mock_req, mock_sess) == True
print("✓ require_role working")

# Test permissions
admin_user = {'role': 'admin'}
mod_user = {'role': 'moderator'}
reg_user = {'role': 'user'}

# Admin has all permissions
assert has_permission(admin_user, 'any_permission') == True
assert has_permission(admin_user, 'delete_users') == True

# Moderator has specific permissions
assert has_permission(mod_user, 'moderate_content') == True
assert has_permission(mod_user, 'read_all_data') == True
assert has_permission(mod_user, 'delete_users') == False

# Regular user has limited permissions
assert has_permission(reg_user, 'read_own_data') == True
assert has_permission(reg_user, 'moderate_content') == False
print("✓ Permission checks working")

# Test require_permission
mock_sess['user'] = admin_user
assert require_permission('any_permission', mock_req, mock_sess) == True

mock_sess['user'] = mod_user  
assert require_permission('moderate_content', mock_req, mock_sess) == True
assert require_permission('delete_users', mock_req, mock_sess) == False
print("✓ require_permission working")

print("\n✅ All core function tests passed!")

Testing core permission functions...
✓ require_auth working
✓ Role hierarchy working
✓ require_role working
✓ Permission checks working
✓ require_permission working

✅ All core function tests passed!


In [ ]:
#| hide
# Test permission management
print("Testing permission management...")

# Debug: Check if PERMISSION_DESCRIPTIONS exists and its content
print(f"PERMISSION_DESCRIPTIONS exists: {'PERMISSION_DESCRIPTIONS' in globals()}")
if 'PERMISSION_DESCRIPTIONS' in globals():
    print(f"PERMISSION_DESCRIPTIONS type: {type(PERMISSION_DESCRIPTIONS)}")
    print(f"PERMISSION_DESCRIPTIONS content: {PERMISSION_DESCRIPTIONS}")

# Test registering permissions
register_permission('test_permission', 'A test permission')
print(f"After register_permission, PERMISSION_DESCRIPTIONS: {PERMISSION_DESCRIPTIONS}")
assert 'test_permission' in PERMISSION_DESCRIPTIONS
assert PERMISSION_DESCRIPTIONS['test_permission'] == 'A test permission'
print("✓ Permission registration working")

# Test role permission management
original_perms = get_permissions_for_role('user').copy()

# Add permission
add_role_permission('user', 'new_permission')
assert 'new_permission' in get_permissions_for_role('user')

# Remove permission  
remove_role_permission('user', 'new_permission')
assert 'new_permission' not in get_permissions_for_role('user')

# Set permissions
set_role_permissions('custom_role', ['perm1', 'perm2', 'perm3'])
custom_perms = get_permissions_for_role('custom_role')
assert len(custom_perms) == 3
assert 'perm1' in custom_perms
print("✓ Role permission management working")

# Restore original permissions
ROLE_PERMISSIONS['user'] = original_perms
del ROLE_PERMISSIONS['custom_role']

print("\n✅ All permission management tests passed!")

Testing permission management...
PERMISSION_DESCRIPTIONS exists: True
PERMISSION_DESCRIPTIONS type: <class 'dict'>
PERMISSION_DESCRIPTIONS content: {'read_own_data': "Read user's own data", 'update_own_data': "Update user's own data", 'delete_own_data': "Delete user's own data", 'read_all_data': "Read all users' data", 'moderate_content': 'Moderate user-generated content', 'manage_users': 'Create, update, delete other users', 'delete_users': 'Delete user accounts', 'view_sensitive_data': 'View sensitive system data', 'export_data': 'Export system data to CSV', 'view_analytics': 'View analytics dashboard'}
After register_permission, PERMISSION_DESCRIPTIONS: {'read_own_data': "Read user's own data", 'update_own_data': "Update user's own data", 'delete_own_data': "Delete user's own data", 'read_all_data': "Read all users' data", 'moderate_content': 'Moderate user-generated content', 'manage_users': 'Create, update, delete other users', 'delete_users': 'Delete user accounts', 'view_sensiti

In [ ]:
#| hide
# Test decorators
print("Testing decorators...")

from fasthtml.common import RedirectResponse, HTMLResponse

# Create mock objects for this test
mock_req = type('Request', (), {})()
mock_sess = {}

# Test auth_required decorator
@auth_required
def protected_route(req, sess):
    return "Protected content"

# Test with unauthenticated session
mock_sess = {}
result = protected_route(mock_req, mock_sess)
assert isinstance(result, RedirectResponse)
assert result.status_code == 303
print("✓ auth_required redirects when not authenticated")

# Test with authenticated session
mock_sess = {'auth': True, 'user': {'username': 'test'}}
result = protected_route(mock_req, mock_sess)
assert result == "Protected content"
print("✓ auth_required allows authenticated users")

# Test role_required decorator  
@role_required('admin')
def admin_route(req, sess):
    return "Admin content"

# Test with no auth
mock_sess = {}
result = admin_route(mock_req, mock_sess)
assert isinstance(result, RedirectResponse)
print("✓ role_required redirects when not authenticated")

# Test with wrong role
mock_sess = {'auth': True, 'user': {'role': 'user'}}
result = admin_route(mock_req, mock_sess)
assert isinstance(result, HTMLResponse)
assert result.status_code == 403
print("✓ role_required returns 403 for insufficient role")

# Test with correct role
mock_sess = {'auth': True, 'user': {'role': 'admin'}}
result = admin_route(mock_req, mock_sess)
assert result == "Admin content"
print("✓ role_required allows users with correct role")

# Test permission_required decorator
@permission_required('delete_users')
def delete_route(req, sess):
    return "Delete content"

# Test with permission
mock_sess = {'auth': True, 'user': {'role': 'admin'}}  # Admin has all permissions
result = delete_route(mock_req, mock_sess)
assert result == "Delete content"
print("✓ permission_required allows users with permission")

# Test without permission
mock_sess = {'auth': True, 'user': {'role': 'user'}}
result = delete_route(mock_req, mock_sess)
assert isinstance(result, HTMLResponse)
assert result.status_code == 403
print("✓ permission_required returns 403 for insufficient permission")

print("\n✅ All decorator tests passed!")

In [ ]:
#| hide
# Test _extract_req_sess helper
print("Testing _extract_req_sess helper...")

def test_func(req, sess, other_arg):
    pass

# Create mock objects for this test
req_obj = type('Request', (), {})()
sess_obj = {'test': 'session'}

# Test positional args
req, sess = _extract_req_sess(test_func, (req_obj, sess_obj, 'other'), {})
assert req == req_obj
assert sess == sess_obj
print("✓ Extracts from positional args")

# Test keyword args
req, sess = _extract_req_sess(test_func, (), {'req': req_obj, 'sess': sess_obj, 'other_arg': 'other'})
assert req == req_obj
assert sess == sess_obj  
print("✓ Extracts from keyword args")

# Test mixed args
req, sess = _extract_req_sess(test_func, (req_obj,), {'sess': sess_obj, 'other_arg': 'other'})
assert req == req_obj
assert sess == sess_obj
print("✓ Extracts from mixed args")

print("\n✅ All helper tests passed!")

### Interactive Demo

In [ ]:
#| eval: false
# Complete demo app showing all permission features
from fasthtml.common import *
from launch_kit.auth import user_auth_before
from launch_kit.permissions import *

# Configure auth beforeware
beforeware = Beforeware(
    user_auth_before,
    skip=['/login', '/public', '/']
)

app, rt = fast_app(before=beforeware)

# Mock user database
users = {
    'admin@example.com': {'id': 1, 'email': 'admin@example.com', 'role': 'admin'},
    'mod@example.com': {'id': 2, 'email': 'mod@example.com', 'role': 'moderator'},
    'user@example.com': {'id': 3, 'email': 'user@example.com', 'role': 'user'}
}

# Public home page
@rt('/')
def get():
    return Div(
        H1("Permissions Demo"),
        P("Test different user roles:"),
        Ul(
            Li("admin@example.com - Admin role"),
            Li("mod@example.com - Moderator role"),
            Li("user@example.com - User role")
        ),
        A("Login", href="/login", cls="button")
    )

# Login page
@rt('/login')
def get():
    return Div(
        H2("Login"),
        Form(
            Input(name="email", type="email", placeholder="Email", required=True),
            Button("Login", type="submit"),
            method="post"
        )
    )

@rt('/login', methods=['POST'])
def post(req, sess, email: str):
    if email in users:
        sess['auth'] = True
        sess['user'] = users[email]
        return RedirectResponse('/dashboard', status_code=303)
    return "Invalid email"

# User dashboard - requires authentication
@rt('/dashboard')
@auth_required
def get(req, sess):
    user = get_user_from_session(sess)
    permissions = get_user_permissions(user)
    
    return Div(
        H1(f"Welcome {user['email']}"),
        P(f"Role: {user['role']}"),
        H3("Your Permissions:"),
        Ul(*[Li(perm) for perm in sorted(permissions)]) if '*' not in permissions else P("All permissions"),
        H3("Test Areas:"),
        Ul(
            Li(A("Admin Area", href="/admin")),
            Li(A("Moderator Area", href="/moderate")),
            Li(A("Delete Users", href="/users/delete"))
        ),
        A("Logout", href="/logout")
    )

# Admin only area
@rt('/admin')
@role_required('admin')
def get(req, sess):
    return Div(
        H1("Admin Area"),
        P("Only admins can see this!"),
        A("Back to Dashboard", href="/dashboard")
    )

# Moderator area (admins can also access)
@rt('/moderate')
@role_required('moderator')
def get(req, sess):
    user = get_user_from_session(sess)
    return Div(
        H1("Moderator Area"),
        P(f"Welcome {user['role']}! Moderators and admins can see this."),
        A("Back to Dashboard", href="/dashboard")
    )

# Permission-based access
@rt('/users/delete')
@permission_required('delete_users')
def get(req, sess):
    return Div(
        H1("Delete Users"),
        P("This requires the 'delete_users' permission."),
        P("Only admins have this by default."),
        A("Back to Dashboard", href="/dashboard")
    )

# Logout
@rt('/logout')
def get(sess):
    sess.clear()
    return RedirectResponse('/', status_code=303)

# Run the demo
from fasthtml.jupyter import JupyUvi
server = JupyUvi(app)

In [ ]:
#| eval: false
# View the app right here in the notebook by uncommenting the line below
from fasthtml.jupyter import HTMX
# HTMX()

In [ ]:
#| eval: false
# Stop the server gracefully
# Note: Always run this after testing to clean up otherwise there will be a dangling thread
# https://fastht.ml/docs/tutorials/jupyter_and_fasthtml.html#graceful-shutdowns
print("Stopping server...")
server.stop()

Stopping server...


## Best Practices

### 1. **Use Decorators for Clean Code**
```python
# Good: Clean and declarative
@rt('/admin')
@role_required('admin')
def get(req, sess):
    return admin_panel()

# Avoid: Manual checks in every route
@rt('/admin')
def get(req, sess):
    if not require_role('admin', req, sess):
        return RedirectResponse('/login')  
    return admin_panel()
```

### 2. **Role Hierarchy for Flexibility**
```python
# Admins automatically get access to moderator areas
@role_required('moderator')  # Admins can also access
```

### 3. **Granular Permissions for Sensitive Operations**
```python
# Use specific permissions for dangerous operations
@permission_required('delete_all_data')  # More specific than @role_required('admin')
```

### 4. **Clear Permission Cache After Role Changes**
```python
def promote_to_admin(user_id, sess):
    # Update user role in database
    update_user_role(user_id, 'admin')
    # Clear cached permissions
    clear_permission_cache(sess)
```

### 5. **Combine with Beforeware for Global Auth**
```python
# Use beforeware for site-wide auth
beforeware = Beforeware(user_auth_before, skip=['/login', '/public'])

# Then use decorators for specific permissions
@role_required('admin')
```

## Security Considerations

### 🔒 Permission Design

| Practice | Implementation |
|----------|----------------|
| **Principle of Least Privilege** | Give users minimum required permissions |
| **Role Separation** | Don't combine unrelated permissions |
| **Audit Trail** | Log permission changes and access attempts |
| **Regular Review** | Periodically review role assignments |

### 🛡️ Implementation Security

| Practice | Implementation |
|----------|----------------|
| **Session Security** | Use secure session configuration |
| **CSRF Protection** | Verify state-changing operations |
| **Rate Limiting** | Limit permission check attempts |
| **Error Handling** | Don't leak permission info in errors |

In [ ]:
#| hide
# Clean up test modifications
if 'test_permission' in PERMISSION_DESCRIPTIONS:
    del PERMISSION_DESCRIPTIONS['test_permission']

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Summary

Launch Kit's permissions module provides:

- **Simple API** - Boolean functions and clean decorators
- **Role Hierarchy** - Admins can access moderator areas automatically
- **Granular Permissions** - Beyond roles for specific operations
- **FastHTML Native** - Works seamlessly with req/sess patterns
- **Transparent** - No hidden middleware or magic
- **Flexible** - Easy to extend with custom roles and permissions
- **Performance** - Optional session caching for efficiency

The module follows Launch Kit's philosophy of being simple, transparent, and flexible while providing all the features needed for production applications.